In [32]:
%matplotlib inline
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.query_api import QueryApi
import matplotlib.pyplot as plt
import numpy as np

In [33]:
token = 'MlUmh8wWKDAcBiZbBjvJPguGKxoON1ZRqFjVQ_zh8M3NHLcr53WnxbfXX_ByD0Wm7S8G0kb622gj2EX25MzFCA=='
org = "Portfolio_Opt"
url = "http://localhost:8080"
bucket = "Nifty50"

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

In [34]:
symbols = ['ASIANPAINT.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'HCLTECH.NS', 'INFY.NS', 'TCS.NS', 'TECHM.NS']

start_date = '2023-01-01'  
end_date = '2024-01-01'
all_data = pd.DataFrame()

In [35]:
for symbol in symbols:
    # Construct the Flux query
    query = f'''
    from(bucket: "{bucket}")
      |> range(start: {start_date}, stop: {end_date})
      |> filter(fn: (r) => r["_measurement"] == "market_data")
      |> filter(fn: (r) => r["_field"] == "close")
      |> filter(fn: (r) => r["symbol"] == "{symbol}")
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    '''
    # Execute the query
    result = query_api.query_data_frame(query=query)
    if not result.empty:
        result.set_index('_time', inplace=True)
        result = result[['close']]  # Ensure only close data is used
        result.columns = [symbol]  # Rename column to symbol name

        if all_data.empty:
            all_data = result
        else:
            all_data = all_data.join(result, how='outer')

print(all_data)


                           ASIANPAINT.NS  BAJAJ-AUTO.NS  BAJFINANCE.NS  \
_time                                                                    
2023-01-02 00:00:00+00:00    3047.250000    3573.949951    6553.750000   
2023-01-03 00:00:00+00:00    3028.250000    3601.699951    6606.250000   
2023-01-04 00:00:00+00:00    3016.850098    3552.850098    6571.000000   
2023-01-05 00:00:00+00:00    3004.350098    3621.149902    6100.049805   
2023-01-06 00:00:00+00:00    2978.399902    3642.250000    5980.399902   
...                                  ...            ...            ...   
2023-12-22 00:00:00+00:00    3341.300049    6372.100098    7295.299805   
2023-12-26 00:00:00+00:00    3383.350098    6464.549805    7162.299805   
2023-12-27 00:00:00+00:00    3404.449951    6709.649902    7232.799805   
2023-12-28 00:00:00+00:00    3397.250000    6703.299805    7258.649902   
2023-12-29 00:00:00+00:00    3402.399902    6797.250000    7327.750000   

                            HCLTECH.N

In [36]:
returns=[]
for symbol in symbols:
    returns.append((all_data[symbol].iloc[-1]-all_data[symbol][0])/all_data[symbol][0])

print(returns)

[np.float64(0.11654767490155057), np.float64(0.9018872935731027), np.float64(0.11810032424184627), np.float64(0.41045749622967187), np.float64(0.012401590822875657), np.float64(0.1631022885942921), np.float64(0.26067362497678304)]


/var/folders/wr/mslv0h8d73n9kkgc30xk05lm0000gn/T/ipykernel_24433/3194192809.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  returns.append((all_data[symbol].iloc[-1]-all_data[symbol][0])/all_data[symbol][0])


In [37]:
weights=[0.1966473 , 0.13984502, 0.21313214 ,0.06534316, 0.10815051, 0.18365964,0.09322223]

portfolio_return=np.dot(returns,weights)

print(portfolio_return)

0.25663191948906605


In [38]:
nifty_data=yf.download('^NSEI', start=start_date, end=end_date)
nifty_return=(nifty_data['Close'].iloc[-1]-nifty_data['Close'][0])/nifty_data['Close'][0]
print(nifty_return)

[*********************100%%**********************]  1 of 1 completed

0.1942003590389879



/var/folders/wr/mslv0h8d73n9kkgc30xk05lm0000gn/T/ipykernel_24433/1972782699.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nifty_return=(nifty_data['Close'].iloc[-1]-nifty_data['Close'][0])/nifty_data['Close'][0]
